# CIFAR vectorized LiRA example

This notebook mirrors A_cifar_main.ipynb but configures the LiRA attack to use the vectorized fast path.

It is safer to delete previous results for target and shadow models as it will train new models. 


In [1]:
import os
import sys
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.insert(0, project_root)

train_config_path = "train_robust.yaml"
audit_config_path = "audit_robust.yaml"


Next, we create the population dataset by concatenating the train and test data. To create the population, we make use of the UserDataset provided in the InputHandler.

In [2]:
from torchvision.datasets import CIFAR10, CIFAR100
from torch import cat, tensor
import pickle
from cifar_handler import CifarInputHandler

# Load the config.yaml file
with open(train_config_path, 'r') as file:
    train_config = yaml.safe_load(file)

data_root = train_config["data"]["data_dir"]
data_path = os.path.join(os.getcwd(), data_root)

# Load the CIFAR train and test datasets
if train_config["data"]["dataset"] == "cifar10":
    trainset = CIFAR10(root=data_root, train=True, download=True)
    testset = CIFAR10(root=data_root, train=False, download=True)
elif train_config["data"]["dataset"] == "cifar100":
    trainset = CIFAR100(root=data_root, train=True, download=True)
    testset = CIFAR100(root=data_root, train=False, download=True)
else:
    raise ValueError("Unknown dataset type")

train_data = tensor(trainset.data).permute(0, 3, 1, 2).float() / 255  # (N, C, H, W)
test_data = tensor(testset.data).permute(0, 3, 1, 2).float() / 255

# Ensure train and test data looks correct
assert train_data.shape[0] == 50000, "Train data should have 50000 samples"
assert test_data.shape[0] == 10000, "Test data should have 10000 samples"
assert train_data.shape[1] == 3, "Data should have 3 channels"
assert test_data.shape[1] == 3, "Data should have 3 channels"
assert train_data.max() <= 1 and train_data.min() >= 0, "Data should be normalized"
assert test_data.max() <= 1 and test_data.min() >= 0, "Data should be normalized"

# Concatenate train and test data into the population
data = cat([train_data.clone().detach(), test_data.clone().detach()], dim=0)
targets = cat([tensor(trainset.targets), tensor(testset.targets)], dim=0)
# Create UserDataset object
population_dataset = CifarInputHandler.UserDataset(data, targets)

assert len(population_dataset) == 60000, "Population dataset should have 60000 samples"

# Store the population dataset to be used by LeakPro 
dataset_name = train_config["data"]["dataset"]
file_path =  data_root+"/"+ dataset_name + ".pkl"
if not os.path.exists(file_path):
    with open(file_path, "wb") as file:
        pickle.dump(population_dataset, file)
        print(f"Save data to {file_path}")
        


With the population dataset stored, we next create the train and test set that will go in to training the target model.

In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import numpy as np

train_fraction = train_config["data"]["f_train"]
test_fraction = train_config["data"]["f_test"]
batch_size = train_config["train"]["batch_size"]

dataset_size = len(population_dataset)
train_size = int(train_fraction * dataset_size) 
test_size = int(test_fraction * dataset_size)
print(f"original train size: {len(train_data)}, test size: {len(test_data)}")
print(f"selected train size: {train_size}, test size: {test_size}")

randomize_audit = False
selected_index = np.arange(train_size + test_size)
if randomize_audit:
    np.random.shuffle(selected_index)
train_indices = selected_index[:train_size]
test_indices = selected_index[train_size:(train_size+test_size)]

# Now define the new target train and test sets
train_subset = CifarInputHandler.UserDataset(data[train_indices], targets[train_indices])
test_subset = CifarInputHandler.UserDataset(data[test_indices], targets[test_indices], **train_subset.return_params())

train_loader = DataLoader(train_subset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_subset, batch_size = batch_size, shuffle = False)

# Evaluate mean and variance of the train data
#train_mean = train_subset.mean
#train_std = train_subset.std
#print (f"Train mean: {train_mean}")
#print (f"Train std: {train_std}")


original train size: 50000, test size: 10000
selected train size: 30000, test size: 30000


### Train target model
With the train and test dataloader in place, we train a ResNet18. 
After training, we call LeakPro to create metadata that will be used during auditing.

In [4]:
target_dir = train_config["run"]["log_dir"]
print("target_dir:", target_dir)


target_dir: ./robust/target


In [ ]:
from torch import save, optim, nn
from target_model_class import ResNet18, WideResNet

# Train the model
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
    
if train_config["data"]["dataset"] == "cifar10":
    num_classes = 10
elif train_config["data"]["dataset"] == "cifar100":
    num_classes = 100
else:
    raise ValueError("Invalid dataset name")

# Create instance of target model
#model = ResNet18(num_classes = num_classes)
model =  WideResNet(depth=28, num_classes=num_classes, widen_factor=2)

# Read out the relevant parameters for training
lr = train_config["train"]["learning_rate"]
weight_decay = train_config["train"]["weight_decay"]
momentum = train_config["train"]["momentum"]
epochs = train_config["train"]["epochs"]
    
# Create optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# train target model
train_result = CifarInputHandler().train(dataloader=train_loader,
                            model=model,
                            criterion=criterion,
                            optimizer=optimizer,
                            epochs=epochs)


Epoch 1/25: 100%|██████████| 106/106 [00:08<00:00, 11.81it/s, acc=0.3943, loss=1.7106]


Validation loss at epoch 1: 1.5474


Epoch 2/25: 100%|██████████| 106/106 [00:08<00:00, 12.53it/s, acc=0.5584, loss=1.2296]


Validation loss at epoch 2: 1.1909


Epoch 3/25: 100%|██████████| 106/106 [00:08<00:00, 12.73it/s, acc=0.6414, loss=1.0012]


...

Epoch 11/25: 100%|██████████| 106/106 [00:08<00:00, 12.83it/s, acc=0.9441, loss=0.1630]


Validation loss at epoch 11: 0.8239


Epoch 12/25: 100%|██████████| 106/106 [00:08<00:00, 12.63it/s, acc=0.9560, loss=0.1281]


Validation loss at epoch 12: 0.8383


Epoch 13/25:  96%|█████████▌| 102/106 [00:08<00:00, 12.58it/s, acc=0.9780, loss=0.0721]

In [ ]:
# Evaluate on test set
test_result = CifarInputHandler().eval(test_loader, model, criterion)

# Store the model and metadata
model = train_result.model
model.to("cpu")
#with open(train_config["run"]["log_dir"]+"/target_model.pkl", "wb") as f:
with open(target_dir+"/target_model.pkl", "wb") as f:
    save(model.state_dict(), f)

# Create metadata to be used by LeakPro
from leakpro import LeakPro
meta_data = LeakPro.make_mia_metadata(train_result = train_result,
                                      optimizer = optimizer,
                                      loss_fn = criterion,
                                      dataloader = train_loader,
                                      test_result = test_result,
                                      epochs = epochs,
                                      train_indices = train_indices,
                                      test_indices = test_indices,
                                      dataset_name = dataset_name)

with open(target_dir+"/model_metadata.pkl", "wb") as f:
    pickle.dump(meta_data, f)
    

Plot accuracy and test of training and test.

In [ ]:
import matplotlib.pyplot as plt

train_acc = train_result.metrics.extra["accuracy_history"]
train_loss = train_result.metrics.extra["loss_history"]
test_acc = test_result.accuracy
test_loss = test_result.loss

# Plot training and test accuracy
plt.figure(figsize=(5, 4))

plt.subplot(1, 2, 1)
plt.plot(train_acc, label='Train Accuracy')
plt.plot(len(train_loss)-1, test_acc, 'ro', label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

# Plot training and test loss
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(len(train_loss)-1, test_loss, 'ro', label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

### Train shadow models

In [ ]:
from leakpro.schemas import LeakProConfig

with open(audit_config_path, "rb") as f:
    audit_configs = yaml.safe_load(f)
leakpro_configs = LeakProConfig(**audit_configs)


In [ ]:
shadow_dir = audit_configs["audit"]["output_dir"]
print("shadow_dir:", shadow_dir)


In [ ]:
import numpy as np

from leakpro.attacks.mia_attacks.lira import AttackLiRA
from leakpro.attacks.utils.shadow_model_handler import ShadowModelHandler
#from shadow_model_handler import ShadowModelHandler # less verbose
from leakpro.input_handler.mia_handler import MIAHandler

from cifar_handler import CifarInputHandler

# Create the handler, it mostly serves as a dummy handler needed to init the attack but it also creates a dataloader,
# contains the metadata for creating the attack and how shadow models are created for example.

handler = MIAHandler(leakpro_configs, CifarInputHandler)

#configs = handler.configs.audit.attack_list[0] 
for configs in handler.configs.audit.attack_list:
    if configs["attack"] == "lira" and configs["online"]== True:        
        print(f'found online lira attack')
        attack = AttackLiRA(handler=handler, configs=configs)
        break
    else:
        print(f'attack {configs["attack"]} is not lira')
        attack = None

assert configs["attack"]=='lira', "ERROR: attack must be lira"
assert configs["online"]== True, "ERROR: attack must online"


In [ ]:
# Get attack data indices. If true, the whole population is used, otherwise exclude the training data.
attack_data_indices = attack.sample_indices_from_population(include_train_indices = True, include_test_indices = True)

training_data_fraction = attack.training_data_fraction # Use the same fraction as the target model


In [ ]:
#Set number of shadow models to train
num_shadow_models = configs["num_shadow_models"]

# Train shadow models
#print("Note that incremental is set to ", INCREMENTAL)
#print("Note that shuffle_shift is set to ", SHUFFLE_SHIFT)
shadow_model_indices = ShadowModelHandler(handler).create_shadow_models(num_models = num_shadow_models,
                                                                shadow_population =  attack_data_indices,
                                                                training_fraction = training_data_fraction,
                                                                online = True, 
                                                                #verbose = False,
                                                                #incremental = INCREMENTAL, 
                                                                #shuffle_shift = SHUFFLE_SHIFT
                                                                       )


In [ ]:
# Get the signal function from the attack like this
#signal = attack.signal

# Set the batch size we want to use for the signal function to extract signals faster
attack.handler.dataloader_config.params["batch_size"] = 1024

# Extract our logits
#target_models_logits = np.swapaxes(signal([attack.target_model],
#                                    attack.handler,
#                                    audit_dataset["data"],
#                                ), 0, 1).squeeze()

#print("target_models_logits", target_models_logits.shape)

attack.prepare_attack()

print(attack.shadow_models_logits.shape, attack.target_logits.shape)


In [ ]:
# Get the audit dataset from the attack
audit_dataset = attack.audit_dataset
audit_data_indices = audit_dataset["data"]

# Reconstruct the audit data indices array
target_audit_data_indices = np.concatenate([train_indices, test_indices])
assert all(audit_data_indices == target_audit_data_indices)


In [ ]:
# Now create in indices mask for target model compliant with those for shadow
target_in_indices_mask = np.zeros_like(target_audit_data_indices)
target_in_indices_mask[:len(train_indices)] = 1
target_in_indices_mask = target_in_indices_mask.astype(bool)
target_in_indices_mask


In [ ]:
# Get in indices mask for shadow models 

in_indices_masks = ShadowModelHandler(handler).get_in_indices_mask(shadow_model_indices, audit_dataset["data"])#.astype(int)


In [ ]:
filepath = target_dir + "/signals"
os.makedirs(filepath, exist_ok=True)

print("store target results in ", filepath)

np.save(filepath + "/rescaled_logits.npy", attack.target_logits[:,None]) # make it 2D array
np.save(filepath + "/in_indices_masks.npy", target_in_indices_mask[:,None]) #  make it 2D array
np.save(filepath + "/audit_data_indices.npy", target_audit_data_indices)

print("attack.target_logits", attack.target_logits.shape)
print("target_in_indices_mask", target_in_indices_mask.shape)


In [ ]:
filepath = shadow_dir + "/signals"
os.makedirs(filepath, exist_ok=True)

print("store shadow results in ", filepath)

np.save(filepath + "/rescaled_logits.npy", attack.shadow_models_logits.T)
np.save(filepath + "/in_indices_masks.npy", in_indices_masks)
np.save(filepath + "/audit_data_indices.npy", audit_data_indices)

print("shadow_models_logits", attack.shadow_models_logits.shape)
print("in_indices_masks", in_indices_masks.shape)
